In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# Define helper function to standardize strings
def safe_upper_strip(x):
    if isinstance(x, str):
        return x.upper().strip()
    else:
        return np.nan

# -------------------------------------------------------
# Step 1: Load and Preprocess TIES Data
# -------------------------------------------------------
ties_df = pd.read_excel("TIESv4-1.xls")
print("TIES Columns:", ties_df.columns)

# Convert targetstate to numeric Country_Code
ties_df['Country_Code'] = pd.to_numeric(ties_df['targetstate'], errors='coerce')
ties_df = ties_df.dropna(subset=['Country_Code'])
ties_df['Country_Code'] = ties_df['Country_Code'].astype(int)

# Convert startyear to numeric
ties_df['startyear'] = pd.to_numeric(ties_df['startyear'], errors='coerce')
ties_df = ties_df.dropna(subset=['startyear'])
ties_df['startyear'] = ties_df['startyear'].astype(int)

# Create Sanctions_Imposed from imposition
if 'imposition' in ties_df.columns:
    ties_df['Sanctions_Imposed'] = ties_df['imposition'].apply(lambda x: 1 if x == 1 else 0)
else:
    raise ValueError("The TIES dataset does not contain 'imposition' column.")

# -------------------------------------------------------
# Step 2: Load COW Data for Country Mapping
# -------------------------------------------------------
cow = pd.read_excel("cow.xlsx")
cow['StateNme_standardized'] = cow['StateNme'].apply(safe_upper_strip)
country_mapping = dict(zip(cow['StateNme_standardized'], cow['CCode']))

# -------------------------------------------------------
# Step 3: Load Polity Data (which contains autocratic scores)
# -------------------------------------------------------
# We assume 'POLITY5-PRC.xlsx' is the Polity dataset file that you mentioned contains the autocratic data.
# Adjust the filename if necessary.
polity_df = pd.read_excel("POLITY5-PRC.xlsx")

# Inspect columns in Polity data
print("Polity Columns:", polity_df.columns)
print(polity_df.head())

# From your previous code, we know that the autocratic scores are spread across multiple year columns.
# Identify which columns represent years:
year_columns = [col for col in polity_df.columns if col.isdigit()]

# Melt the Polity dataset to long format with 'Year' and 'Value'
# Assuming 'Economy Name' is the identifier column as per your previous code snippet.
if 'Economy Name' not in polity_df.columns:
    raise ValueError("Polity data does not have 'Economy Name' column. Adjust the code accordingly.")

melted_df_auto = polity_df.melt(
    id_vars=['Economy Name'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Convert 'Year' to numeric and drop rows where Year or Value is missing
melted_df_auto['Year'] = pd.to_numeric(melted_df_auto['Year'], errors='coerce')
melted_df_auto = melted_df_auto.dropna(subset=['Year'])
melted_df_auto['Year'] = melted_df_auto['Year'].astype(int)

melted_df_auto['Value'] = pd.to_numeric(melted_df_auto['Value'], errors='coerce')
melted_df_auto = melted_df_auto.dropna(subset=['Value'])

# Standardize economy names and map to Country_Code
melted_df_auto['Economy_Name_standardized'] = melted_df_auto['Economy Name'].apply(safe_upper_strip)
melted_df_auto['Country_Code'] = melted_df_auto['Economy_Name_standardized'].map(country_mapping)
melted_df_auto = melted_df_auto.dropna(subset=['Country_Code'])
melted_df_auto['Country_Code'] = melted_df_auto['Country_Code'].astype(int)

print("melted_df_auto Columns:", melted_df_auto.columns)
print(melted_df_auto.head())

# -------------------------------------------------------
# Step 4: Merge TIES Data with Autocracy Data
# -------------------------------------------------------
df_with_autocracy = pd.merge(
    ties_df,
    melted_df_auto[['Country_Code', 'Year', 'Value']],
    left_on=['Country_Code', 'startyear'],
    right_on=['Country_Code', 'Year'],
    how='left'
)

# Rename Value to Autocracy_Score
df_with_autocracy.rename(columns={'Value': 'Autocracy_Score'}, inplace=True)

# Drop the redundant 'Year' column
df_with_autocracy.drop(columns=['Year'], inplace=True)

print("df_with_autocracy Columns:", df_with_autocracy.columns)
print(df_with_autocracy.head())

# -------------------------------------------------------
# Step 5: Choose Features and Target
# -------------------------------------------------------
# Ensure these columns exist. Adjust if they do not:
# 'threat', 'targetcosts', 'sendercosts', 'Autocracy_Score', 'Sanctions_Imposed'
required_columns = ['threat', 'targetcosts', 'sendercosts', 'Autocracy_Score', 'Sanctions_Imposed']
for col in required_columns:
    if col not in df_with_autocracy.columns:
        print(f"Column '{col}' not found in df_with_autocracy. Available columns:")
        print(df_with_autocracy.columns)
        raise ValueError("Adjust feature selection based on the actual dataset.")

features = ['threat', 'targetcosts', 'sendercosts', 'Autocracy_Score']
X = df_with_autocracy[features]
y = df_with_autocracy['Sanctions_Imposed']

# -------------------------------------------------------
# Step 6: Handle Missing Values
# -------------------------------------------------------
df_with_autocracy = df_with_autocracy.dropna(subset=['Sanctions_Imposed'])
X = X.fillna(X.mean())

# Ensure no missing rows remain
df_with_autocracy = df_with_autocracy.dropna(subset=features + ['Sanctions_Imposed'])
X = df_with_autocracy[features]
y = df_with_autocracy['Sanctions_Imposed']

# -------------------------------------------------------
# Step 7: Scale and Split Data
# -------------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

# -------------------------------------------------------
# Step 8: Train Logistic Regression Model
# -------------------------------------------------------
model = LogisticRegression()
model.fit(X_train, y_train)

# -------------------------------------------------------
# Step 9: Evaluation
# -------------------------------------------------------
y_pred = model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Feature importance
feature_importances = pd.DataFrame(model.coef_[0], index=features, columns=["Importance"]).sort_values(by="Importance", ascending=False)
print("Feature Importance:\n", feature_importances)

# (Optional) Plot feature importances
plt.figure(figsize=(8,4))
plt.bar(feature_importances.index, feature_importances['Importance'], color='skyblue')
plt.title('Feature Importances')
plt.xlabel('Features')
plt.ylabel('Coefficient Importance')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


TIES Columns: Index(['caseid', 'startmonth', 'startday', 'startyear', 'endmonth', 'endday',
       'endyear', 'ongoingasofmonth', 'ongoingasofday', 'ongoingasofyear',
       'sender1', 'sender2', 'sender3', 'sender4', 'sender5', 'primarysender',
       'targetstate', 'institution', 'institutionid', 'targetinstitution',
       'otherinstitution', 'issue1', 'issue2', 'issue3', 'otherissue',
       'threat', 'threatid1', 'threatid2', 'threatid3', 'sanctiontypethreat',
       'othersanctiontypethreatened', 'bspecif', 'scommit',
       'threatenedtargetinterest', 'dsanctions', 'carrots',
       'anticipatedtargetcosts', 'anticipatedtargetcostsfigureifav',
       'tcurrency', 'tyearofestimate', 'anticipatedsendercosts', 'scurrency',
       'syearofestimate', 'imposition', 'sancimpositionstartmonth',
       'sancimpositionstartday', 'sancimpositionstartyear', 'sanctionidentity',
       'sanctiontype', 'othersanctiontype', 'implementationofdiplomaticsancti',
       'carrotsduringsanction', 'ca

FileNotFoundError: [Errno 2] No such file or directory: 'cow.xlsx'

In [5]:
print(ties_df.columns)


Index(['caseid', 'startmonth', 'startday', 'startyear', 'endmonth', 'endday',
       'endyear', 'ongoingasofmonth', 'ongoingasofday', 'ongoingasofyear',
       'sender1', 'sender2', 'sender3', 'sender4', 'sender5', 'primarysender',
       'targetstate', 'institution', 'institutionid', 'targetinstitution',
       'otherinstitution', 'issue1', 'issue2', 'issue3', 'otherissue',
       'threat', 'threatid1', 'threatid2', 'threatid3', 'sanctiontypethreat',
       'othersanctiontypethreatened', 'bspecif', 'scommit',
       'threatenedtargetinterest', 'dsanctions', 'carrots',
       'anticipatedtargetcosts', 'anticipatedtargetcostsfigureifav',
       'tcurrency', 'tyearofestimate', 'anticipatedsendercosts', 'scurrency',
       'syearofestimate', 'imposition', 'sancimpositionstartmonth',
       'sancimpositionstartday', 'sancimpositionstartyear', 'sanctionidentity',
       'sanctiontype', 'othersanctiontype', 'implementationofdiplomaticsancti',
       'carrotsduringsanction', 'carrotvalue', 'c